In [5]:
%store -r

In [6]:
import boto3
import botocore.exceptions
import time

In [7]:
# Get event source mapppings UUID

lambda_client = boto3.client('lambda')

try:
    list_event_source_mappings_response = lambda_client.list_event_source_mappings(
        FunctionName = LambdaFunctionName,
        EventSourceArn=MSKClusterArn
    )
    for mapping in list_event_source_mappings_response['EventSourceMappings']:
        print('Event source mapping UUID:', mapping['UUID'])
        event_source_mapping_uuid = mapping['UUID']
except botocore.exceptions.ClientError as error:
    print(error)
    raise error

Event source mapping UUID: 491b1ceb-6e02-445d-9d44-4b7715966a56


In [8]:
# Delete the event source mapping

lambda_client = boto3.client('lambda')

try:
    delete_event_source_mapping_response = lambda_client.delete_event_source_mapping(
        UUID = event_source_mapping_uuid
    )
    print(delete_event_source_mapping_response)
except botocore.exceptions.ClientError as error:
    print(error)
    raise error

{'ResponseMetadata': {'RequestId': '83c9dff1-80a8-460c-981c-c22f0e29a749', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Thu, 23 Jan 2025 19:55:07 GMT', 'content-type': 'application/json', 'content-length': '1280', 'connection': 'keep-alive', 'x-amzn-requestid': '83c9dff1-80a8-460c-981c-c22f0e29a749'}, 'RetryAttempts': 0}, 'UUID': '491b1ceb-6e02-445d-9d44-4b7715966a56', 'StartingPosition': 'LATEST', 'BatchSize': 100, 'EventSourceArn': 'arn:aws:kafka:us-east-1:825222715469:cluster/BedrockStreamIngestTest3MSKCluster/cf6bf71b-132e-4d2b-be15-f171515967ad-25', 'FunctionArn': 'arn:aws:lambda:us-east-1:825222715469:function:BedrockStreamIngestTest3KafkaConsumerLambdaFunction', 'LastModified': datetime.datetime(2025, 1, 23, 19, 55, 7, 183000, tzinfo=tzlocal()), 'LastProcessingResult': 'OK', 'State': 'Deleting', 'StateTransitionReason': 'USER_INITIATED', 'Topics': ['streamtopic'], 'AmazonManagedKafkaEventSourceConfig': {'ConsumerGroupId': '491b1ceb-6e02-445d-9d44-4b7715966a56'}}


In [23]:
# Ensure the event source mapping id deleted

lambda_client = boto3.client('lambda')
mapping_check = 'active'

max_time = time.time() + 1*60*60 # 1 hour

while time.time() < max_time:
    try:
        list_event_source_mappings_response = lambda_client.list_event_source_mappings(
            FunctionName = LambdaFunctionName,
            EventSourceArn=MSKClusterArn
        )
        if not list_event_source_mappings_response['EventSourceMappings']:
            mapping_check = 'inactive'
        else:
            mapping_check = 'active'
    except botocore.exceptions.ClientError as error:
        print(error)
        raise error

    if mapping_check == "inactive":
        print('Event source mapping deleted')
        break
    else:
        print('Event source mapping deleting')
    time.sleep(30)

Event source mapping deleted
